In [2]:
%load_ext autoreload
%autoreload 2

In [87]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio
import warnings

_ = load_dotenv(find_dotenv())
nest_asyncio.apply()
warnings.filterwarnings('ignore')

from llama_index.core import Settings
from llama_index.core.tools.tool_spec.base import BaseToolSpec
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding

import yfinance as yf
import numpy as np
import pandas as pd
from typing import Optional, Literal, List, Union, Tuple

from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.volume import AccDistIndexIndicator
from ta.trend import MACD, AroonIndicator, IchimokuIndicator
from ta.momentum import StochRSIIndicator, StochasticOscillator

In [117]:
class TechnicalAnalyst(BaseToolSpec):
    """These tools are intended for technical analysis and investment recommendations by agents"""
    
    spec_functions = [
        "analyse"
    ]
    
    def __init__(self):
        """Initialize technical analyst tool"""
        
    def get_stock_data(self, 
                       ticker: str,
                       period: Optional[
                           Literal["1d",
                                   "5d",
                                   "1mo",
                                   "3mo",
                                   "6mo",
                                   "1y",
                                   "2y",
                                   "5y",
                                   "10y",
                                   "ytd",
                                   "max"]
                           ] = "10y") -> pd.DataFrame:
        """Gets the daily historical prices and volume for a ticker across a specified period"""
        return yf.Ticker(ticker).history(period=period)

    def get_bollinger_bands(self,
                            df: pd.DataFrame, 
                            column: str = "Close",
                            window: int = 20,
                            window_dev: int = 2
                            ):
        """The Bollinger Bands are a volatility indicator of the price for an asset in a specific period of time. 
        There are 3 bands, the Middle Band (MB) is the average of the price in the last n periods, the Upper (UB) and 
        Lower Bands (LB) are equal to the middle band, but adding and subtracting x times the standard deviation.
        
        When the closing price surpasses the upper or lower bands, there are sudden changes in the price. It is usually 
        a good idea to sell when it is higher than the Upper Band and to buy when it is lower than the Lower Band. 
        While valuable, Bollinger Bands are a secondary indicator that is best used to confirm other analysis methods
        
        The column of interest is the "Close" column.
        
        Args:
            window: number of periods. The usual is 20
            window_dev: factor of standard deviations. The usual is 2
        
        """
        
        indicator_bb = BollingerBands(close=df[column], window=window, window_dev=window_dev)
        # Add Bollinger Bands features
        df['bb_bbm'] = indicator_bb.bollinger_mavg()
        df['bb_bbh'] = indicator_bb.bollinger_hband()
        df['bb_bbl'] = indicator_bb.bollinger_lband()

        # Add Bollinger Band high indicator
        df['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

        # Add Bollinger Band low indicator
        df['bb_bbli'] = indicator_bb.bollinger_lband_indicator()
        
        # The recommendation        
        df['bb_recommendation'] = np.where(
            (df['Close'] < df['bb_bbl']) & (df['Close']<=df['bb_bbh']),
            "buy",
            np.where(
                (df['Close'] > df['bb_bbl']) & (df['Close']>=df['bb_bbh']),
                "sell",
                "wait"
            )
        )
        df['bb_explanation'] = np.where(
            df['bb_recommendation'] == "buy",
            "The closing price is below the low Bollinger band",
            np.where(
                df['bb_recommendation'] == "sell",
                "The closing price is above the high Bollinger band",
                "The closing price is within the low and high Bollinger bands"
            )
        )
        return df

    def get_macd(self,
                 df: pd.DataFrame,
                 window_fast: int = 12,
                 window_slow: int = 26,
                 window_sign: int = 9,
                 ):
        """Moving Average Convergence Divergence Is a trend-following momentum indicator that shows the relationship 
        between two moving averages of prices.
        
        When the MACD is smaller than the MACD signal or when the MACD difference has a value lower than zero, it indicates
        that the price trend will be bearish. The contrary represents a price increase.
        
        The column of interest is the "Close" column.
        
        Reference:
        https://www.investopedia.com/terms/m/macd.asp
        """
        df_ = MACD(df['Close'],
                   window_fast=window_fast,
                   window_slow=window_slow,
                   window_sign=window_sign)
        df['macd'] = df_.macd()
        df['macd_diff'] = df_.macd_diff()
        df['macd_signal'] = df_.macd_signal()
        
        df['macd_recommendation'] = np.where(
            (df['macd'] < df['macd_signal']), #bearish price signal
            "sell",
            np.where(
                (df['macd'] > df['macd_signal']) & (df['macd']>0), #bullish price signal
                "buy",
                np.where(
                    (df['macd'] > df['macd_signal']) & (df['macd']<0), #short trade
                    "short",
                    "wait"
                )
            )
        )
        df['macd_explanation'] = np.where(
            df['macd_recommendation'] == "sell",
            "The MACD curve is beneath the MACD signal curve",
            np.where(
                df['macd_recommendation'] == "buy",
                "The MACD curve is above the MACD signal curve and the MACD values are greater than 0",
                np.where(
                    df['macd_recommendation'] == "short",
                    "The MACD curve is above the MACD signal curve, and the MACD values are lower than 0",
                    "No clear signal that the market is overbought or oversold."
                )
            )
        )  
        return df

    def get_stoch_rsi(self,
                      df: pd.DataFrame,
                      window: int = 14,
                      smooth1: int = 3,
                      smooth2: int = 3):
        """
        The stochastic RSI applies the stochastic oscillator formula to a set of 
        relative strength index (RSI) values instead of standard price data.
        
        The RSI indicator gauges momentum and trend strength
        1. It compares the recent price gains vs recent price losses
        2. When the RSI is above 70, the asset is considered overbought and could decline
        3. When the RSI is below 30, the asset is oversold and could rally.
        4. When the indicator is moving in a different direction than the price, it shows
        that the current price trend is weakening and could soon reverse.
        
        The column of interest is the "Close" column
        
        StochRSI deems something to be oversold when the value drops below 0.20 and an
        upward price movement is possible. A value above 0.80 suggests the RSI is at an
        extreme high and could be used to signal a pullback.
        
        When the StochRSI is above 0.50, the security may be seen as trending higher.
        
        Being a 2nd derivative of price (the RSI is a 1st derivative of price), the 
        stoch RSI moves faster and is more sensitive to price changes.
        
        smooth1 and smooth2 are the smoothing windows for the %K(fast) and %(D) slow
        stochastic oscillators. %K represents the percentage difference between the highest
        and lowest values of the security over a time period. %D represents the smooth2
        period average of %K and is used to show longer term trends.
        
        Column of interest: Close
        """
        df_ = StochRSIIndicator(df['Close'],
                                window = window,
                                smooth1 = smooth1,
                                smooth2 = smooth2)
        df['stochrsi'] = df_.stochrsi()
        df['stochrsi_recommendation'] = np.where(
            df['stochrsi']<0.2,
            "buy",
            np.where(
                df['stochrsi'] > 0.8,
                "sell",
                "wait"
            )
        )
        df['stochrsi_explanation'] = np.where(
            df['stochrsi_recommendation']=="buy",
            "The stochastic RSI value is lower than 0.2 indicating that the security is possibly oversold",
            np.where(
                df['stochrsi_recommendation']=="sell",
                "The stochastic RSI value is above 0.8 indicating that the security is potentially overbought",
                "The stochastic RSI value indicates that the security is neither oversold nor overbought"
            )
        )
        return df
    
    def get_stoch_oscillator(self,
                             df: pd.DataFrame,
                             window: int = 14,
                             smooth_window: int = 3):
        """Applies a stochastic operator formula onto prices
        The output of %K (fast) is a percentage difference between the highest and lowest
        values of the security over a time period (window). The signal (%D) is a smoothed
        period average of %K to show longer term trends.
        
        Columns of interest: High, low, close
        """
        df_=StochasticOscillator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                window = window,
                smooth_window=smooth_window
            )
        df['stoch_signal'] = df_.stoch_signal()
        df['stoch_pct_change'] = df['stoch_signal'].pct_change()
        df['stoch_trend'] = np.where(
            df['stoch_pct_change'] > 0,
            '+',
            '-'
        )
        df['stoch_recommendation'] = np.where(
            df['stoch_trend']=='+',
            'buy',
            'sell'
        )
        df['stoch_explanation'] = np.where(
            df['stoch_recommendation'] == 'buy',
            f"The {smooth_window} smoothed stochastic indicator period trend of the security is positive",
            f"The {smooth_window} smoothed stochastic indicator period trend of the security is negative"
        )
        return df
    
    def get_aroon_indicator(self,
                            df: pd.DataFrame,
                            window: int = 25):
        """
        The Aroon Indicator measures whether a security is in a trend, specifically whether the
        price is hitting new highs or lows over the calculation period. When the Aroon up crosses
        the Aroon down, that is the first sign of a possible trend change. If the Aroon hits 100
        and stays relatively close to that level while the Aroon down stays near zero, this is 
        a positive confirmation of an uptrend. The converse is true.
        
        Columns of interest: High, Low
        """
        df_ = AroonIndicator(
            high = df['High'],
            low = df['Low'],
            window = window
        )
        df['aroon_indicator'] = df_.aroon_indicator()
        df['aroon_indicator_pct_change'] = df['aroon_indicator'].pct_change()
        df['aroon_sign_change'] = np.sign(df['aroon_indicator']).diff().ne(0)
        df['aroon_recommendation'] = np.where(
            (df['aroon_indicator']>0) & (df['aroon_sign_change'] == True) \
            & (df['aroon_indicator_pct_change'] > 0),
            "buy",
            np.where(
                (df['aroon_indicator']<0) & (df['aroon_sign_change'] == True) \
                & (df['aroon_indicator_pct_change'] < 0),
                "sell",
                "wait")  
        )
        df['aroon_explanation'] = np.where(
            df['aroon_recommendation']=="buy",
            "The aroon indicator is positive indicating that aroon up is above aroon down. This is also the start of a trend change moment",
            np.where(
                df['aroon_recommendation']=="sell",
                "The aroon indicator is negative indicating that aroon up is below aroon down. This is also the start of a trend change moment",
                "No further indication of trend changes. Wait"
            )
        )
        return df
    
    def get_AccDistIndex(self,
                         df: pd.DataFrame):
        """
        Accumulation/Distribution lines accounts for the trading range for the period, and where
        the close is in relation to that range (including the closing price of that period).
        
        If a stock finishes near its high, the indicator gives volume more weightage. If the indicator
        line trends up, the stock closes above the halfway point of the range which indicates buying 
        interest. The converse is true.
        
        If the A/D starts falling while the price rises, this signals that the price trend could reverse.
        
        Columns of interest: High, Low, Close, Volume
        """
        df_ = AccDistIndexIndicator(
            high = df['High'],
            low = df['Low'],
            close = df['Close'],
            volume = df['Volume']
        )
        df['acc_dist_index'] = df_.acc_dist_index()
        df['adi_pct_change'] = df['acc_dist_index'].pct_change()
        df['adi_trend'] = np.where(
            df['adi_pct_change']>0,
            "+",
            "-"
        )
        df['close_pct_change'] = df['Close'].pct_change()
        df['close_trend'] = np.where(
            df['Close']>0,
            '+',
            '-'
        )
        df['adi_recommendation'] = np.where(
            (df['adi_trend'] == "+") & (df['close_trend']=="+"),
            "buy",
            np.where(
                (df['adi_trend']=="+") & (df['close_trend']=="-"),
                "wait",
                np.where(
                    (df['adi_trend']=="-") & (df['close_trend']=="-"),
                    "sell",
                    "wait"
                )
            )
        )
        df['adi_explanation'] = np.where(
            df['adi_recommendation']=="buy",
            "The accumulation/distribution index trends suggests it's good to buy",
            np.where(
                df['adi_recommendation'] =="sell",
                "The accumulation/distribution index trends suggests it's good to sell",
                "The accumulation/distribution index trend conflicts with price trends, suggesting it's good to wait"
            )
        )
        return df
    
    def get_ichimoku_indicator(self,
                               df: pd.DataFrame,
                               window1: int = 9,
                               window2: int = 26,
                               window3: int = 52):
        """
        The Ichimoku Cloud is composed of 5 lines or calculations, 2 of which comprise a 
        'cloud' where the difference between the 2 lines is shaded in. The lines include
        a 9 period average, 26 period average, an average of both averages, a 52 period
        average and a lagging closing part line.
        
        The cloud is key. When the price is below the cloud, the trend is down, and vice 
        versa. The trends are strengthened if the cloud is moving in the same direction as
        the price.
        
        Columns of interest: High, Low, Close
        """
        df_ = IchimokuIndicator(
            high = df['High'],
            low = df['Low'],
            window1=window1,
            window2=window2,
            window3=window3
        )
        df['ichimoku_a'] = df_.ichimoku_a()
        df['ichimoku_b'] = df_.ichimoku_b()
        df['ichimoku_cloud_indicator'] = np.where(
            df['ichimoku_a'] - df['ichimoku_b'] > 0,
            "green",
            "red"
        )
        df['ichimoku_recommendation'] = np.where(
            (df['Close']>df['ichimoku_a']) & (df['Close']>df['ichimoku_b']) \
            & (df['ichimoku_cloud_indicator'] == 'green'),
            "buy",
            np.where(
                (df['Close']<df['ichimoku_a']) & (df['Close']<df['ichimoku_b']) \
                & (df['ichimoku_cloud_indicator'] == 'red'),
                "sell",
                "wait"
            )
        )
        df['ichimoku_explanation'] = np.where(
            df['ichimoku_recommendation']=="buy",
            "The price is above the cloud. The stock price is in uptrend.",
            np.where(
                df['ichimoku_recommendation']=="sell",
                "The price is below the cloud. The stock price is in downtrend.",
                "The price trend is in transition. Wait."
            )
        )
        return df
    
    def analyse(self, 
                ticker: str,
                period: Optional[
                           Literal["1d",
                                   "5d",
                                   "1mo",
                                   "3mo",
                                   "6mo",
                                   "1y",
                                   "2y",
                                   "5y",
                                   "10y",
                                   "ytd",
                                   "max"]
                           ] = "10y") -> pd.DataFrame:
        """
        Super function to conduct technical analysis on a particular stock ticker.
        The analysis methods are:
        1. Accumulation/Distribution Index ("adi"): A trend metric of stock price and volume
        2. Aroon Indicator: Measures whether securities are in trend and if a new trend is beginning
        3. Bollinger Bands: An indication of stock price volatility and momentum by determining where 
        prices are relative to each other.
        4. Ichimoku indicator: Measures whether price and averaged average price trends are coherent
        5. Moving average convergence divergence ("macd"): Ascertains trend direction and momentum
        6. Stochastic Oscillator ("stoch"): Measures the current price relative to the price range over
        a number of periods
        7. Stochastic Relative Strength Index ("stochrsi"): A 2nd derivative trend indicator of price
        focusing on market momentum. This index is more sensitive than the relative
        strengh index
        """
        df = self.get_stock_data(ticker=ticker, period=period)
        df = self.get_AccDistIndex(df=df)
        df = self.get_aroon_indicator(df=df)
        df = self.get_bollinger_bands(df=df)
        df = self.get_ichimoku_indicator(df=df)
        df = self.get_macd(df=df)
        df = self.get_stoch_oscillator(df=df)
        df = self.get_stoch_rsi(df=df)
        rec_columns = [column for column in df.columns if "recommendation" in column]
        expl_columns = [column for column in df.columns if "explanation" in column]
        # columns.append("Close")
        df_rec = df[rec_columns].tail(1).transpose()
        df_rec = df_rec.reset_index()
        df_rec.columns = ["field", "recommendation"]
        df_expl = df[expl_columns].tail(1).transpose()
        df_expl = df_expl.reset_index()
        df_expl.columns = ["field", "elaboration"]
        df_rec['elaboration'] = df_expl['elaboration']
        return df_rec

In [118]:
ta = TechnicalAnalyst()
df= ta.analyse(ticker="ilmn")

In [119]:
df

,field,recommendation,elaboration
0,adi_recommendation,wait,The accumulation/distribution index trend conf...
1,aroon_recommendation,wait,No further indication of trend changes. Wait
2,bb_recommendation,wait,The closing price is within the low and high B...
3,ichimoku_recommendation,wait,The price trend is in transition. Wait.
4,macd_recommendation,short,"The MACD curve is above the MACD signal curve,..."
5,stoch_recommendation,buy,The 3 smoothed stochastic indicator period tre...
6,stochrsi_recommendation,wait,The stochastic RSI value indicates that the se...


In [120]:
ta_tool_list = ta.to_tool_list()

In [121]:
Settings.llm = BedrockConverse(
    model = "anthropic.claude-3-haiku-20240307-v1:0",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name = os.environ["AWS_DEFAULT_REGION"]
)
Settings.embed_model = BedrockEmbedding(
    model = "amazon.titan-embed-text-v1",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_region_name = os.environ["AWS_DEFAULT_REGION"]
)

In [129]:
from llama_index.core.agent import (
    FunctionCallingAgentWorker,
    AgentRunner
)
from IPython.display import display, Markdown

agent_worker = FunctionCallingAgentWorker.from_tools(
    tools = ta_tool_list,
    llm = Settings.llm,
    verbose = True)
agent = AgentRunner(agent_worker=agent_worker)

In [130]:
query = "Conduct a technical analysis on Illumina shares. Should I buy Illumina shares?"
response = agent.chat(query)

display(Markdown(f"<b>{response}</b>"))

Added user message to memory: Conduct a technical analysis on Illumina shares. Should I buy Illumina shares?
=== LLM Response ===
Okay, let's perform a technical analysis on Illumina (ticker: ILMN) shares to help determine if it's a good investment:
=== Calling Function ===
Calling function: analyse with args: {"ticker": "ILMN", "period": "1y"}
=== Function Output ===
                     field recommendation  \
0       adi_recommendation            buy   
1     aroon_recommendation           wait   
2        bb_recommendation           wait   
3  ichimoku_recommendation           wait   
4      macd_recommendation          short   
5     stoch_recommendation            buy   
6  stochrsi_recommendation           wait   

                                         elaboration  
0  The accumulation/distribution index trends sug...  
1       No further indication of trend changes. Wait  
2  The closing price is within the low and high B...  
3            The price trend is in transition. W

<b>Based on the technical analysis, here's a summary of the key findings for Illumina (ILMN) shares:

- Accumulation/Distribution Index (ADI): Suggests a buy signal, indicating the stock price is being accumulated.
- Aroon Indicator: Suggests a neutral/wait signal, with no clear indication of a new trend emerging.
- Bollinger Bands: Suggests a neutral/wait signal, as the current price is within the Bollinger Bands.
- Ichimoku Indicator: Suggests a neutral/wait signal, as the price trend is in transition.
- MACD: Suggests a short signal, as the MACD line is above the signal line, indicating potential downward momentum.
- Stochastic Oscillator: Suggests a buy signal, as the stochastic lines are in a bullish crossover.
- Stochastic RSI: Suggests a neutral/wait signal, as the stochastic RSI value indicates the security is neither overbought nor oversold.

Overall, the technical indicators present a mixed picture for Illumina shares. The ADI and stochastic oscillator suggest a buy signal, while the MACD indicates a potential short signal. The other indicators suggest a neutral/wait stance.

Given the mixed signals, I would recommend further fundamental analysis and research into the company's financials, growth prospects, and competitive positioning before making an investment decision on Illumina shares. The technical analysis alone does not provide a clear buy or sell recommendation at this time.</b>

In [131]:
query = "Conduct a technical analysis on NVIDIA shares. Should I buy NVIDIA shares?"
response = agent.chat(query)

display(Markdown(f"<b>{response}</b>"))

Added user message to memory: Conduct a technical analysis on NVIDIA shares. Should I buy NVIDIA shares?
=== LLM Response ===
Okay, let's perform a technical analysis on NVIDIA (ticker: NVDA) shares to help determine if it's a good investment:
=== Calling Function ===
Calling function: analyse with args: {"ticker": "NVDA", "period": "1y"}
=== Function Output ===
                     field recommendation  \
0       adi_recommendation           wait   
1     aroon_recommendation           wait   
2        bb_recommendation           wait   
3  ichimoku_recommendation            buy   
4      macd_recommendation           sell   
5     stoch_recommendation            buy   
6  stochrsi_recommendation           wait   

                                         elaboration  
0  The accumulation/distribution index trend conf...  
1       No further indication of trend changes. Wait  
2  The closing price is within the low and high B...  
3  The price is above the cloud. The stock price ...  

<b>Based on the technical analysis, here's a summary of the key findings for NVIDIA (NVDA) shares:

- Accumulation/Distribution Index (ADI): Suggests a neutral/wait signal, as the ADI trend is not conclusive.
- Aroon Indicator: Suggests a neutral/wait signal, with no clear indication of a new trend emerging.
- Bollinger Bands: Suggests a neutral/wait signal, as the current price is within the Bollinger Bands.
- Ichimoku Indicator: Suggests a buy signal, as the price is above the Ichimoku cloud, indicating a bullish trend.
- MACD: Suggests a sell signal, as the MACD line is below the signal line, indicating potential downward momentum.
- Stochastic Oscillator: Suggests a buy signal, as the stochastic lines are in a bullish crossover.
- Stochastic RSI: Suggests a neutral/wait signal, as the stochastic RSI value indicates the security is neither overbought nor oversold.

The technical indicators present a mixed picture for NVIDIA shares. The Ichimoku and stochastic oscillator suggest a buy signal, while the MACD indicates a potential sell signal. The other indicators suggest a neutral/wait stance.

Given the mixed signals, I would recommend further fundamental analysis and research into the company's financials, growth prospects, and competitive positioning before making an investment decision on NVIDIA shares. The technical analysis alone does not provide a clear buy or sell recommendation at this time.</b>

In [133]:
query = "Conduct a technical analysis on Amgen shares. Should I buy Amgen shares?"
response = agent.chat(query)

display(Markdown(f"<b>{response}</b>"))

Added user message to memory: Conduct a technical analysis on Amgen shares. Should I buy Amgen shares?
=== LLM Response ===
Okay, let's perform a technical analysis on Amgen (ticker: AMGN) shares to help determine if it's a good investment:
=== Calling Function ===
Calling function: analyse with args: {"ticker": "AMGN", "period": "1y"}
=== Function Output ===
                     field recommendation  \
0       adi_recommendation            buy   
1     aroon_recommendation           wait   
2        bb_recommendation           wait   
3  ichimoku_recommendation           wait   
4      macd_recommendation           sell   
5     stoch_recommendation            buy   
6  stochrsi_recommendation            buy   

                                         elaboration  
0  The accumulation/distribution index trends sug...  
1       No further indication of trend changes. Wait  
2  The closing price is within the low and high B...  
3            The price trend is in transition. Wait.  
4 

<b>Based on the technical analysis, here's a summary of the key findings for Amgen (AMGN) shares:

- Accumulation/Distribution Index (ADI): Suggests a buy signal, indicating the stock price is being accumulated.
- Aroon Indicator: Suggests a neutral/wait signal, with no clear indication of a new trend emerging.
- Bollinger Bands: Suggests a neutral/wait signal, as the current price is within the Bollinger Bands.
- Ichimoku Indicator: Suggests a neutral/wait signal, as the price trend is in transition.
- MACD: Suggests a sell signal, as the MACD line is below the signal line, indicating potential downward momentum.
- Stochastic Oscillator: Suggests a buy signal, as the stochastic lines are in a bullish crossover.
- Stochastic RSI: Suggests a buy signal, as the stochastic RSI value is below 0.2, indicating the security is oversold.

The technical indicators present a mixed picture for Amgen shares. The ADI, stochastic oscillator, and stochastic RSI suggest a buy signal, while the MACD indicates a potential sell signal. The other indicators suggest a neutral/wait stance.

Given the mixed signals, I would recommend further fundamental analysis and research into Amgen's financials, growth prospects, and competitive positioning before making an investment decision. The technical analysis alone does not provide a clear buy or sell recommendation at this time.

Ultimately, the decision to invest in Amgen shares should be based on your own risk tolerance, investment goals, and a comprehensive analysis of the company's overall outlook. The technical analysis provides useful insights, but should be considered alongside other factors.</b>

In [134]:
query = "Elaborate on the Ichimoku cloud recommendations on Amgen shares"
response = agent.chat(query)

display(Markdown(f"<b>{response}</b>"))

Added user message to memory: Elaborate on the Ichimoku cloud recommendations on Amgen shares
=== LLM Response ===
Okay, let's take a closer look at the Ichimoku indicator recommendation for Amgen (AMGN) shares:
=== Calling Function ===
Calling function: analyse with args: {"ticker": "AMGN", "period": "1y"}
=== Function Output ===
                     field recommendation  \
0       adi_recommendation            buy   
1     aroon_recommendation           wait   
2        bb_recommendation           wait   
3  ichimoku_recommendation           wait   
4      macd_recommendation           sell   
5     stoch_recommendation            buy   
6  stochrsi_recommendation            buy   

                                         elaboration  
0  The accumulation/distribution index trends sug...  
1       No further indication of trend changes. Wait  
2  The closing price is within the low and high B...  
3            The price trend is in transition. Wait.  
4    The MACD curve is beneath 

<b>The Ichimoku indicator recommendation for Amgen (AMGN) shares is a "wait" signal.

The Ichimoku indicator is a versatile technical analysis tool that looks at multiple components to assess the overall trend and momentum of a stock. The key components are:

1. Tenkan-sen (Conversion Line): The average of the highest high and lowest low over the past 9 periods.
2. Kijun-sen (Base Line): The average of the highest high and lowest low over the past 26 periods.
3. Senkou Span A (Leading Span A): The average of the Tenkan-sen and Kijun-sen, plotted 26 periods ahead.
4. Senkou Span B (Leading Span B): The average of the highest high and lowest low over the past 52 periods, plotted 26 periods ahead.
5. Chikou Span (Lagging Span): The current closing price plotted 26 periods behind.

The "wait" signal in this case indicates that the Ichimoku components are not providing a clear directional signal for Amgen's stock price. The price is likely in a transitional phase, where the trend is not firmly established.

Specifically, the "wait" recommendation suggests that:
- The current price is not decisively above or below the Ichimoku cloud (the area between Senkou Span A and B)
- The Tenkan-sen and Kijun-sen lines are not providing a clear bullish or bearish crossover signal
- The Chikou Span is not clearly above or below the price action

This neutral/transitional signal from the Ichimoku indicator means the trend is unclear, and more time or additional confirmation may be needed to determine the next likely direction for Amgen's stock price. The "wait" recommendation advises caution and further analysis before making an investment decision.</b>

## Export

In [1]:
%%writefile ../tools/technical_analysis_tools.py

import warnings
warnings.filterwarnings('ignore')

from llama_index.core.tools.tool_spec.base import BaseToolSpec
import yfinance as yf

import numpy as np
import pandas as pd
from typing import Optional, Literal, List, Union, Tuple

from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.volume import AccDistIndexIndicator
from ta.trend import MACD, AroonIndicator, IchimokuIndicator
from ta.momentum import StochRSIIndicator, StochasticOscillator

class TechnicalAnalyst(BaseToolSpec):
    """These tools are intended for technical analysis and investment recommendations by agents"""
    
    spec_functions = [
        "analyse"
    ]
    
    def __init__(self):
        """Initialize technical analyst tool"""
        
    def get_stock_data(self, 
                       ticker: str,
                       period: Optional[
                           Literal["1d",
                                   "5d",
                                   "1mo",
                                   "3mo",
                                   "6mo",
                                   "1y",
                                   "2y",
                                   "5y",
                                   "10y",
                                   "ytd",
                                   "max"]
                           ] = "10y") -> pd.DataFrame:
        """Gets the daily historical prices and volume for a ticker across a specified period"""
        return yf.Ticker(ticker).history(period=period)

    def get_bollinger_bands(self,
                            df: pd.DataFrame, 
                            column: str = "Close",
                            window: int = 20,
                            window_dev: int = 2
                            ):
        """The Bollinger Bands are a volatility indicator of the price for an asset in a specific period of time. 
        There are 3 bands, the Middle Band (MB) is the average of the price in the last n periods, the Upper (UB) and 
        Lower Bands (LB) are equal to the middle band, but adding and subtracting x times the standard deviation.
        
        When the closing price surpasses the upper or lower bands, there are sudden changes in the price. It is usually 
        a good idea to sell when it is higher than the Upper Band and to buy when it is lower than the Lower Band. 
        While valuable, Bollinger Bands are a secondary indicator that is best used to confirm other analysis methods
        
        The column of interest is the "Close" column.
        
        Args:
            window: number of periods. The usual is 20
            window_dev: factor of standard deviations. The usual is 2
        
        """
        
        indicator_bb = BollingerBands(close=df[column], window=window, window_dev=window_dev)
        # Add Bollinger Bands features
        df['bb_bbm'] = indicator_bb.bollinger_mavg()
        df['bb_bbh'] = indicator_bb.bollinger_hband()
        df['bb_bbl'] = indicator_bb.bollinger_lband()

        # Add Bollinger Band high indicator
        df['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

        # Add Bollinger Band low indicator
        df['bb_bbli'] = indicator_bb.bollinger_lband_indicator()
        
        # The recommendation        
        df['bb_recommendation'] = np.where(
            (df['Close'] < df['bb_bbl']) & (df['Close']<=df['bb_bbh']),
            "buy",
            np.where(
                (df['Close'] > df['bb_bbl']) & (df['Close']>=df['bb_bbh']),
                "sell",
                "wait"
            )
        )
        df['bb_explanation'] = np.where(
            df['bb_recommendation'] == "buy",
            "The closing price is below the low Bollinger band",
            np.where(
                df['bb_recommendation'] == "sell",
                "The closing price is above the high Bollinger band",
                "The closing price is within the low and high Bollinger bands"
            )
        )
        return df

    def get_macd(self,
                 df: pd.DataFrame,
                 window_fast: int = 12,
                 window_slow: int = 26,
                 window_sign: int = 9,
                 ):
        """Moving Average Convergence Divergence Is a trend-following momentum indicator that shows the relationship 
        between two moving averages of prices.
        
        When the MACD is smaller than the MACD signal or when the MACD difference has a value lower than zero, it indicates
        that the price trend will be bearish. The contrary represents a price increase.
        
        The column of interest is the "Close" column.
        
        Reference:
        https://www.investopedia.com/terms/m/macd.asp
        """
        df_ = MACD(df['Close'],
                   window_fast=window_fast,
                   window_slow=window_slow,
                   window_sign=window_sign)
        df['macd'] = df_.macd()
        df['macd_diff'] = df_.macd_diff()
        df['macd_signal'] = df_.macd_signal()
        
        df['macd_recommendation'] = np.where(
            (df['macd'] < df['macd_signal']), #bearish price signal
            "sell",
            np.where(
                (df['macd'] > df['macd_signal']) & (df['macd']>0), #bullish price signal
                "buy",
                np.where(
                    (df['macd'] > df['macd_signal']) & (df['macd']<0), #short trade
                    "short",
                    "wait"
                )
            )
        )
        df['macd_explanation'] = np.where(
            df['macd_recommendation'] == "sell",
            "The MACD curve is beneath the MACD signal curve",
            np.where(
                df['macd_recommendation'] == "buy",
                "The MACD curve is above the MACD signal curve and the MACD values are greater than 0",
                np.where(
                    df['macd_recommendation'] == "short",
                    "The MACD curve is above the MACD signal curve, and the MACD values are lower than 0",
                    "No clear signal that the market is overbought or oversold."
                )
            )
        )  
        return df

    def get_stoch_rsi(self,
                      df: pd.DataFrame,
                      window: int = 14,
                      smooth1: int = 3,
                      smooth2: int = 3):
        """
        The stochastic RSI applies the stochastic oscillator formula to a set of 
        relative strength index (RSI) values instead of standard price data.
        
        The RSI indicator gauges momentum and trend strength
        1. It compares the recent price gains vs recent price losses
        2. When the RSI is above 70, the asset is considered overbought and could decline
        3. When the RSI is below 30, the asset is oversold and could rally.
        4. When the indicator is moving in a different direction than the price, it shows
        that the current price trend is weakening and could soon reverse.
        
        The column of interest is the "Close" column
        
        StochRSI deems something to be oversold when the value drops below 0.20 and an
        upward price movement is possible. A value above 0.80 suggests the RSI is at an
        extreme high and could be used to signal a pullback.
        
        When the StochRSI is above 0.50, the security may be seen as trending higher.
        
        Being a 2nd derivative of price (the RSI is a 1st derivative of price), the 
        stoch RSI moves faster and is more sensitive to price changes.
        
        smooth1 and smooth2 are the smoothing windows for the %K(fast) and %(D) slow
        stochastic oscillators. %K represents the percentage difference between the highest
        and lowest values of the security over a time period. %D represents the smooth2
        period average of %K and is used to show longer term trends.
        
        Column of interest: Close
        """
        df_ = StochRSIIndicator(df['Close'],
                                window = window,
                                smooth1 = smooth1,
                                smooth2 = smooth2)
        df['stochrsi'] = df_.stochrsi()
        df['stochrsi_recommendation'] = np.where(
            df['stochrsi']<0.2,
            "buy",
            np.where(
                df['stochrsi'] > 0.8,
                "sell",
                "wait"
            )
        )
        df['stochrsi_explanation'] = np.where(
            df['stochrsi_recommendation']=="buy",
            "The stochastic RSI value is lower than 0.2 indicating that the security is possibly oversold",
            np.where(
                df['stochrsi_recommendation']=="sell",
                "The stochastic RSI value is above 0.8 indicating that the security is potentially overbought",
                "The stochastic RSI value indicates that the security is neither oversold nor overbought"
            )
        )
        return df
    
    def get_stoch_oscillator(self,
                             df: pd.DataFrame,
                             window: int = 14,
                             smooth_window: int = 3):
        """Applies a stochastic operator formula onto prices
        The output of %K (fast) is a percentage difference between the highest and lowest
        values of the security over a time period (window). The signal (%D) is a smoothed
        period average of %K to show longer term trends.
        
        Columns of interest: High, low, close
        """
        df_=StochasticOscillator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                window = window,
                smooth_window=smooth_window
            )
        df['stoch_signal'] = df_.stoch_signal()
        df['stoch_pct_change'] = df['stoch_signal'].pct_change()
        df['stoch_trend'] = np.where(
            df['stoch_pct_change'] > 0,
            '+',
            '-'
        )
        df['stoch_recommendation'] = np.where(
            df['stoch_trend']=='+',
            'buy',
            'sell'
        )
        df['stoch_explanation'] = np.where(
            df['stoch_recommendation'] == 'buy',
            f"The {smooth_window} smoothed stochastic indicator period trend of the security is positive",
            f"The {smooth_window} smoothed stochastic indicator period trend of the security is negative"
        )
        return df
    
    def get_aroon_indicator(self,
                            df: pd.DataFrame,
                            window: int = 25):
        """
        The Aroon Indicator measures whether a security is in a trend, specifically whether the
        price is hitting new highs or lows over the calculation period. When the Aroon up crosses
        the Aroon down, that is the first sign of a possible trend change. If the Aroon hits 100
        and stays relatively close to that level while the Aroon down stays near zero, this is 
        a positive confirmation of an uptrend. The converse is true.
        
        Columns of interest: High, Low
        """
        df_ = AroonIndicator(
            high = df['High'],
            low = df['Low'],
            window = window
        )
        df['aroon_indicator'] = df_.aroon_indicator()
        df['aroon_indicator_pct_change'] = df['aroon_indicator'].pct_change()
        df['aroon_sign_change'] = np.sign(df['aroon_indicator']).diff().ne(0)
        df['aroon_recommendation'] = np.where(
            (df['aroon_indicator']>0) & (df['aroon_sign_change'] == True) \
            & (df['aroon_indicator_pct_change'] > 0),
            "buy",
            np.where(
                (df['aroon_indicator']<0) & (df['aroon_sign_change'] == True) \
                & (df['aroon_indicator_pct_change'] < 0),
                "sell",
                "wait")  
        )
        df['aroon_explanation'] = np.where(
            df['aroon_recommendation']=="buy",
            "The aroon indicator is positive indicating that aroon up is above aroon down. This is also the start of a trend change moment",
            np.where(
                df['aroon_recommendation']=="sell",
                "The aroon indicator is negative indicating that aroon up is below aroon down. This is also the start of a trend change moment",
                "No further indication of trend changes. Wait"
            )
        )
        return df
    
    def get_AccDistIndex(self,
                         df: pd.DataFrame):
        """
        Accumulation/Distribution lines accounts for the trading range for the period, and where
        the close is in relation to that range (including the closing price of that period).
        
        If a stock finishes near its high, the indicator gives volume more weightage. If the indicator
        line trends up, the stock closes above the halfway point of the range which indicates buying 
        interest. The converse is true.
        
        If the A/D starts falling while the price rises, this signals that the price trend could reverse.
        
        Columns of interest: High, Low, Close, Volume
        """
        df_ = AccDistIndexIndicator(
            high = df['High'],
            low = df['Low'],
            close = df['Close'],
            volume = df['Volume']
        )
        df['acc_dist_index'] = df_.acc_dist_index()
        df['adi_pct_change'] = df['acc_dist_index'].pct_change()
        df['adi_trend'] = np.where(
            df['adi_pct_change']>0,
            "+",
            "-"
        )
        df['close_pct_change'] = df['Close'].pct_change()
        df['close_trend'] = np.where(
            df['Close']>0,
            '+',
            '-'
        )
        df['adi_recommendation'] = np.where(
            (df['adi_trend'] == "+") & (df['close_trend']=="+"),
            "buy",
            np.where(
                (df['adi_trend']=="+") & (df['close_trend']=="-"),
                "wait",
                np.where(
                    (df['adi_trend']=="-") & (df['close_trend']=="-"),
                    "sell",
                    "wait"
                )
            )
        )
        df['adi_explanation'] = np.where(
            df['adi_recommendation']=="buy",
            "The accumulation/distribution index trends suggests it's good to buy",
            np.where(
                df['adi_recommendation'] =="sell",
                "The accumulation/distribution index trends suggests it's good to sell",
                "The accumulation/distribution index trend conflicts with price trends, suggesting it's good to wait"
            )
        )
        return df
    
    def get_ichimoku_indicator(self,
                               df: pd.DataFrame,
                               window1: int = 9,
                               window2: int = 26,
                               window3: int = 52):
        """
        The Ichimoku Cloud is composed of 5 lines or calculations, 2 of which comprise a 
        'cloud' where the difference between the 2 lines is shaded in. The lines include
        a 9 period average, 26 period average, an average of both averages, a 52 period
        average and a lagging closing part line.
        
        The cloud is key. When the price is below the cloud, the trend is down, and vice 
        versa. The trends are strengthened if the cloud is moving in the same direction as
        the price.
        
        Columns of interest: High, Low, Close
        """
        df_ = IchimokuIndicator(
            high = df['High'],
            low = df['Low'],
            window1=window1,
            window2=window2,
            window3=window3
        )
        df['ichimoku_a'] = df_.ichimoku_a()
        df['ichimoku_b'] = df_.ichimoku_b()
        df['ichimoku_cloud_indicator'] = np.where(
            df['ichimoku_a'] - df['ichimoku_b'] > 0,
            "green",
            "red"
        )
        df['ichimoku_recommendation'] = np.where(
            (df['Close']>df['ichimoku_a']) & (df['Close']>df['ichimoku_b']) \
            & (df['ichimoku_cloud_indicator'] == 'green'),
            "buy",
            np.where(
                (df['Close']<df['ichimoku_a']) & (df['Close']<df['ichimoku_b']) \
                & (df['ichimoku_cloud_indicator'] == 'red'),
                "sell",
                "wait"
            )
        )
        df['ichimoku_explanation'] = np.where(
            df['ichimoku_recommendation']=="buy",
            "The price is above the cloud. The stock price is in uptrend.",
            np.where(
                df['ichimoku_recommendation']=="sell",
                "The price is below the cloud. The stock price is in downtrend.",
                "The price trend is in transition. Wait."
            )
        )
        return df
    
    def analyse(self, 
                ticker: str,
                period: Optional[
                           Literal["1d",
                                   "5d",
                                   "1mo",
                                   "3mo",
                                   "6mo",
                                   "1y",
                                   "2y",
                                   "5y",
                                   "10y",
                                   "ytd",
                                   "max"]
                           ] = "10y") -> pd.DataFrame:
        """
        Super function to conduct technical analysis on a particular stock ticker.
        The analysis methods are:
        1. Accumulation/Distribution Index ("adi"): A trend metric of stock price and volume
        2. Aroon Indicator: Measures whether securities are in trend and if a new trend is beginning
        3. Bollinger Bands: An indication of stock price volatility and momentum by determining where 
        prices are relative to each other.
        4. Ichimoku indicator: Measures whether price and averaged average price trends are coherent
        5. Moving average convergence divergence ("macd"): Ascertains trend direction and momentum
        6. Stochastic Oscillator ("stoch"): Measures the current price relative to the price range over
        a number of periods
        7. Stochastic Relative Strength Index ("stochrsi"): A 2nd derivative trend indicator of price
        focusing on market momentum. This index is more sensitive than the relative
        strengh index
        """
        df = self.get_stock_data(ticker=ticker, period=period)
        df = self.get_AccDistIndex(df=df)
        df = self.get_aroon_indicator(df=df)
        df = self.get_bollinger_bands(df=df)
        df = self.get_ichimoku_indicator(df=df)
        df = self.get_macd(df=df)
        df = self.get_stoch_oscillator(df=df)
        df = self.get_stoch_rsi(df=df)
        rec_columns = [column for column in df.columns if "recommendation" in column]
        expl_columns = [column for column in df.columns if "explanation" in column]
        # columns.append("Close")
        df_rec = df[rec_columns].tail(1).transpose()
        df_rec = df_rec.reset_index()
        df_rec.columns = ["field", "recommendation"]
        df_expl = df[expl_columns].tail(1).transpose()
        df_expl = df_expl.reset_index()
        df_expl.columns = ["field", "elaboration"]
        df_rec['elaboration'] = df_expl['elaboration']
        return df_rec

def get_ta_tools():
    ta = TechnicalAnalyst()
    return ta.to_tool_list()

Overwriting technical_analysis_tools.py
